In [1]:
include("main.jl")
using PyCall
using CairoMakie
sepsis_gym = pyimport("custom_sepsis")
using Statistics


/Users/luisastue/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
struct MeansType
    individual_runs::Vector{Vector{Float64}}
    mean_rewards::Vector{Float64}
    smoothed_mean::Vector{Float64}
    smoothed_std::Vector{Float64}
    keys_of_smoothed::Vector{Float64}
end
function calculate_mean_rewards(results)
    rews = Dict()
    means = Dict()
    for (type, run) in results
        if occursin("100", String(type))
            window_size = 6
        else 
            window_size = 20
        end
        all_keys = [sort(collect(keys(model.mean_rewards))) for model in run]
        min_keys = minimum(length.(all_keys))
        ks = all_keys[1][1:min_keys]
        filled_rewards = []
        all_rewards = []
        rews = []
        for model in run
            rewards = [model.mean_rewards[1]]
            for i in 2:min_keys
                key = all_keys[1][i]
                mult_factor = key - all_keys[1][i-1]
                push!(rewards, fill(model.mean_rewards[key], mult_factor)...)
            end
            push!(all_rewards, [model.mean_rewards[k] for k in ks])
            push!(filled_rewards, rewards)
            push!(rews, rewards)
        end
        mean_rewards = mean(all_rewards)
        std_rewards = std(all_rewards)
        means[type] = MeansType( rews,  mean(filled_rewards),  moving_avg(mean_rewards, window_size),  Float64.(moving_avg(std_rewards, window_size)),  Float64.(ks))
    end
    return means
end

calculate_mean_rewards (generic function with 1 method)

In [ ]:
ts = Dict(
    :Simple100 => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/Simple-$i.json") for i in 0:2],
    :Medium100 => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/Medium-$i.json") for i in 0:2],
    :None100 => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/None-$i.json") for i in 0:2],
    :None100P => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/None-small-prior-$i.json") for i in 0:2],
    :Softmax100 => [load_jld("data/mcmc/runs/SoftmaxPPL-$i.jld")  for i in 1:3],
    :SimplePPL100 => [load_jld("data/mcmc/runs/SimplePPL-$i.jld")  for i in 1:3],
    :Simple1 => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/Simple-every-$i.json") for i in 1:3],
    :Medium1 => [sepsis_gym.DirThompsonSampling.load_json("json/dbn/ts/Medium-every-$i.json") for i in 1:3],
    :Softmax1 => [load_jld("data/mcmc/runs/SoftmaxPPL-every-$i.jld")  for i in 1:3],
    :SimplePPL1 => [load_jld("data/mcmc/runs/SimplePPL-every-$i.jld")  for i in 1:3],
)

Dict{Symbol, Vector} with 9 entries:
  :Medium1      => PyObject[PyObject <custom_sepsis.models.dirichlet.evaluate.t…
  :Softmax1     => RunData[RunData("SoftmaxPPL-every-1", :Softmax, ModelData(Dy…
  :SimplePPL1   => RunData[RunData("SimplePPL-every-1", :SimplePPL, ModelData(D…
  :SimplePPL100 => RunData[RunData("SimplePPL-1", :SimplePPL, ModelData(Dynamic…
  :Medium100    => PyObject[PyObject <custom_sepsis.models.dirichlet.evaluate.t…
  :Simple100    => PyObject[PyObject <custom_sepsis.models.dirichlet.evaluate.t…
  :None100      => PyObject[PyObject <custom_sepsis.models.dirichlet.evaluate.t…
  :Softmax100   => RunData[RunData("SoftmaxPPL-1", :Softmax, ModelData(DynamicC…
  :Simple1      => PyObject[PyObject <custom_sepsis.models.dirichlet.evaluate.t…

In [4]:
means = calculate_mean_rewards(ts)

Dict{Any, Any} with 9 entries:
  :Simple1      => MeansType([[-0.63441, -0.63441, -0.60263, -0.58272, -0.64829…
  :Softmax1     => MeansType([[-0.60985, -0.75268, -0.76046, -0.83145, -0.84843…
  :SimplePPL1   => MeansType([[-0.36839, -0.5633, -0.58331, -0.55381, -0.44809,…
  :SimplePPL100 => MeansType([[-0.62715, -0.36589, -0.42002, -0.42002, -0.54821…
  :Medium100    => MeansType([[-0.70404, -0.66129, -0.62742, -0.62742, -0.69871…
  :Simple100    => MeansType([[-0.67862, -0.67333, -0.62492, -0.62492, -0.60324…
  :Medium1      => MeansType([[-0.68342, -0.68342, -0.64269, -0.61833, -0.62749…
  :None100      => MeansType([[-0.68426, -0.7111, -0.65799, -0.65799, -0.6539, …
  :Softmax100   => MeansType([[-0.71519, -0.7079, -0.73134, -0.73134, -0.79105,…

In [5]:
using JSON3

function save_means_to_json(means, directory)
    for (key, value) in means
        file_path = joinpath(directory, "$(key).json")
        json_data = Dict(
            "individual_runs" => value.individual_runs,
            "mean_rewards" => value.mean_rewards,
            "smoothed_mean" => value.smoothed_mean,
            "smoothed_std" => value.smoothed_std,
            "keys_of_smoothed" => value.keys_of_smoothed,
            "name"=> key,
            "info" => Dict(),
        )
        JSON3.write(file_path, json_data)
    end
end

# Save means to JSON files in the "means_json" directory
save_means_to_json(means, "json/ts/")